In [12]:
# Import modules
import auxiliary
import gzip

In [47]:
filepath_wt = "../data/WTconcatenate.csv.gz"
filepath_ki = "../data/KIconcatenate.csv.gz"

df_wt = auxiliary.read_dataframe(filepath_wt, low_memory=False)
df_ki = auxiliary.read_dataframe(filepath_ki, low_memory=False)

In [ ]:
df_wt

In [46]:
print(df_wt.columns)
{
    **dict.fromkeys(['bar', 'delta_bar'], lambda ax: ax.bar)
}
{"Condition": ,
"FileName",
"X",
"Y",
"Coherency100",
'Energy100',
'MeanInt100',
'VarInt100',
'Density100',
'VarDensity100',
'Coherency140',
'Energy140',
'MeanInt140',
'VarInt140',
'Density140',
'VarDensity140',
'Coherency20',
'Energy20',
'MeanInt20',
'VarInt20',
'Density20',
'VarDensity20',
'Coherency60',
'Energy60',
'MeanInt60',
'VarInt60',
'Density60',
'VarDensity60',
'OrientationRef20',
'OrientationRef60',
'OrientationRef100',
'OrientationRef140',
'Dist',
'Angle20',
'Angle60',
'Angle100',
'Angle140',
'Mask',
'Type',
'Cells',
'MinDist',
'MedDist',
'CellArea',
'CellEcc',
'Cells100um',
'MinDist100um',
'MedDist100um',
'CellArea100um',
'CellEcc100um',
'Frac'
print(df_wt["Coherency100"].value_counts())

Index(['Condition', 'FileName', 'X', 'Y', 'Coherency100', 'Energy100',
       'MeanInt100', 'VarInt100', 'Density100', 'VarDensity100',
       'Coherency140', 'Energy140', 'MeanInt140', 'VarInt140', 'Density140',
       'VarDensity140', 'Coherency20', 'Energy20', 'MeanInt20', 'VarInt20',
       'Density20', 'VarDensity20', 'Coherency60', 'Energy60', 'MeanInt60',
       'VarInt60', 'Density60', 'VarDensity60', 'OrientationRef20',
       'OrientationRef60', 'OrientationRef100', 'OrientationRef140', 'Dist',
       'Angle20', 'Angle60', 'Angle100', 'Angle140', 'Mask', 'Type', 'Cells',
       'MinDist', 'MedDist', 'CellArea', 'CellEcc', 'Cells100um',
       'MinDist100um', 'MedDist100um', 'CellArea100um', 'CellEcc100um',
       'Frac'],
      dtype='object')
Coherency100
2.000000e-03    19038
3.000000e-03    14173
1.000000e-03    13469
9.200000e-02    12693
9.300000e-02    12653
                ...  
4.162000e-07        1
2.589000e-09        1
7.767000e-10        1
3.542000e-11        1
9.9

In [39]:
df_ki.iloc[:, 38].value_counts()

Type
CD3    2273788
Ly6    1342185
Name: count, dtype: int64